In [1]:
from utils import get_training_objects, print_params, safe_save_model_for_hf_trainer, langely_pipeline
import data
import os

import torch
import transformers
from transformers import Trainer, AutoModelForCausalLM
from peft import get_peft_model, LoraConfig, TaskType

/Users/Xaver.Davey/langley/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenizer, model, data_module = get_training_objects()

lora_config = LoraConfig(
    r=4,
    target_modules=["q_proj", "v_proj"],
    task_type=TaskType.CAUSAL_LM,
    lora_alpha=8,
    lora_dropout=0.1,
)

lora_training_args = transformers.training_args.TrainingArguments(
    bf16=True,
    num_train_epochs=20.0,
    logging_steps=10,
    learning_rate=1e-4,
    label_names=['labels'],
    per_device_train_batch_size=1,
)

lora_model = get_peft_model(model, lora_config)
lora_model.print_trainable_parameters()

#lora_training_args

Dataset({
    features: ['instruction', 'output'],
    num_rows: 15
})


Map: 100%|██████████| 15/15 [00:00<00:00, 270.60 examples/s]

trainable params: 786,432 || all params: 1,347,258,368 || trainable%: 0.0584


In [3]:
lora_trainer = Trainer(
    model=lora_model,
    args=lora_training_args,
    **data_module,
)
lora_trainer.train()

[2025-03-06 22:42:32,055] [INFO] [real_accelerator.py:222:get_accelerator] Setting ds_accelerator to mps (auto detect)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
W0306 22:42:32.417000 75073 torch/distributed/elastic/multiprocessing/redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.


Step,Training Loss
10,1.719200
20,1.405600
30,1.595900
40,1.171700
50,1.218400
60,0.881500
70,0.906600
80,0.573200
90,0.651600
100,0.493200


TrainOutput(global_step=300, training_loss=0.5307100439071655, metrics={'train_runtime': 765.578, 'train_samples_per_second': 0.392, 'train_steps_per_second': 0.392, 'total_flos': 486752774553600.0, 'train_loss': 0.5307100439071655, 'epoch': 20.0})

In [4]:
lora_trainer.model.merge_adapter()

In [5]:
safe_save_model_for_hf_trainer(lora_trainer, "lora_output")

In [7]:
if os.path.exists("lora_output") and False:
    lora_model = AutoModelForCausalLM.from_pretrained("lora_output", torch_dtype=torch.bfloat16)

In [9]:
res = langely_pipeline(tokenizer, lora_model, "Using the langely programming language, create a function that subtracts a second number from the first only if the second number is less than the first.")
if res[0]:
    print(res[1])
else:
    print(res[2])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.
Python(75788) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


```langely
func subtract_if_less(a, b) {
    let result = a;
    if b < a {
        result = a - b;
    };
    return result;
};

print subtract_if_less(10, 5);
print subtract_if_less(5, 10);
print subtract_if_less(10, 10);
```
32021225472133720833325671055200949339321745584335222225255544333220725753048553322251134983377221111522294934953342217115155437171115421793339413331244932132211111511332222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
